In [10]:
import time
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
import gensim
from gensim import corpora, models, parsing
from gensim.models import LdaModel
import nltk
import pickle
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import *
import numpy as np
from sklearn.neighbors import NearestNeighbors
import pyLDAvis.gensim
import itertools
from scipy.sparse import coo_matrix

In [2]:
nltk.download('wordnet', quiet=True)
np.random.seed(2018)
stemmer = PorterStemmer()

In [3]:
class PandasHelper:

    @staticmethod
    def get_id_from_series(series):
        return int(series.index.tolist()[0])

In [28]:
class LDAModel:
    def __init__(self):
        self.no_below = 5
        self.no_above = 0.2
        self.num_topics = 10
        self.num_of_iterations = 100
        self.passes = 3
        self.minimum_probability = 0.01

    def _lemmatize_stemming(self, text):
        return stemmer.stem(WordNetLemmatizer().lemmatize(text))
    
    def _preprocess(self, text):
        unigrams = []
        for token in gensim.utils.simple_preprocess(text):
            if token not in parsing.preprocessing.STOPWORDS and len(token) > 3:
                unigrams.append(self._lemmatize_stemming(token))
                
        # bi_grams = ['_'.join(b) for b in nltk.bigrams(unigrams)]
        # tri_grams = ['_'.join(t) for t in nltk.trigrams(unigrams)]
        
        return list(itertools.chain(unigrams))
    
    def _savePickleFile(self, fileName, data):
        fileName = f'./{fileName}.pickle'
        mappingFile = open(fileName, 'wb')
        pickle.dump(data, mappingFile)
        mappingFile.close()

    def saveModel(self, lda, corpus, df):
        # Save model output
        lda.save('./model')
        # Save corpus
        self._savePickleFile('corpus', corpus)
        self._savePickleFile('df', df)

    def trainModel(self):
        data = pd.read_json('../data/movies_data.json', orient='split')
        documents = data['content']
        ids = data['id']
        processed_docs = documents.map(self._preprocess)

        print('Start training LDA model...')
        dictionary = gensim.corpora.Dictionary(processed_docs)
        dictionary.filter_extremes(no_below = self.no_below, no_above=self.no_above)
        corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
        
        tf_idf = models.TfidfModel(corpus)
        corpus_tf_idf = tf_idf[corpus]
        
        lda = LdaModel(
            corpus_tf_idf,
            num_topics=self.num_topics,
            id2word=dictionary,
            passes=self.passes,
            iterations=self.num_of_iterations,
            minimum_probability=self.minimum_probability)
        
        index = gensim.similarities.MatrixSimilarity(corpus_tf_idf)
        coo = coo_matrix(index)
        similarity_matrix = np.zeros(((len(ids)), len(ids)))
        
        for i,j,v in zip(coo.row, coo.col, coo.data):
            similarity_matrix[i, j] = 1 if v > 1 else v
            
        df_similarity_matrix = pd.DataFrame(similarity_matrix, index=ids, columns=ids)
        
        # lda_display = pyLDAvis.gensim.prepare(lda, corpus_tf_idf, dictionary, sort_topics=False)
        # pyLDAvis.show(lda_display)
        
        print('Finished training LDA model...')
        
        return lda, corpus, df_similarity_matrix
    
lda_model = LDAModel()
lda, corpus, df = lda_model.trainModel()  # train a LDA model using the assgined corpora
lda_model.saveModel(lda, corpus, df) # save model for recommendations use

Start training LDA model...


Finished training LDA model...


In [32]:
class Recommender():
    def __init__(self):
        self.lda = LdaModel.load('./model')
        self.corpus = self.loadPickleFile('./corpus')
        self.df = self.loadPickleFile('./df')
        self.num_of_recommendation = 10
        
    def loadPickleFile(self, fileName):
        file = open(f'{fileName}.pickle','rb')
        object_file = pickle.load(file)
        return object_file
    
    def recommend(self, movie_id):
        data = pd.read_json('../data/movies_data.json', orient='split')
        start = time.time()

        model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
        model_knn.fit(self.df)

        movie_topics = self.get_movie_topics(movie_id)

        if movie_topics is None:
            return None

        distances, indices = model_knn.kneighbors(movie_topics, n_neighbors=self.num_of_recommendation + 1)
        similarities = 1 - distances.flatten()
        similarities = similarities[1:]
        indices = indices.flatten()
        indices = indices[1:]

        end = time.time()
        print(f'Recommended in: {end - start} s')
        return [{
            'id': PandasHelper.get_id_from_series(self.df.iloc[[indices[index]]]),
            'name': data[data['id'] == PandasHelper.get_id_from_series(self.df.iloc[[indices[index]]])].title.iloc[0],
            'similarity': float(line)
        } for index, line in enumerate(similarities)]

    def get_movie_topics(self, movie_id):
        movie_row = self.df[self.df.index == movie_id]

        if movie_row.empty:
            return None

        row_values = movie_row.values.reshape(1, -1)

        return row_values

In [33]:
recommender = Recommender()

In [34]:
recommender.recommend(480)

Recommended in: 1.4677517414093018 s


[{'id': 1544,
  'name': 'The Lost World: Jurassic Park',
  'similarity': 0.6224342338464117},
 {'id': 117529, 'name': 'Jurassic World', 'similarity': 0.590238744432192},
 {'id': 136016, 'name': 'The Good Dinosaur', 'similarity': 0.5057719744673915},
 {'id': 4519,
  'name': 'The Land Before Time',
  'similarity': 0.5010557089695745},
 {'id': 69278, 'name': 'Land of the Lost', 'similarity': 0.49888860087947273},
 {'id': 1017,
  'name': 'Swiss Family Robinson',
  'similarity': 0.47036480381665435},
 {'id': 4445,
  'name': 'T-Rex: Back to the Cretaceous',
  'similarity': 0.4501327921976248},
 {'id': 3400,
  'name': "We're Back! A Dinosaur's Story",
  'similarity': 0.4393099851682969},
 {'id': 69644,
  'name': 'Ice Age: Dawn of the Dinosaurs',
  'similarity': 0.4387942109108951},
 {'id': 5504,
  'name': 'Spy Kids 2: Island of Lost Dreams',
  'similarity': 0.40948724119046376}]